# Movie Genre Classification with LSTM
## Sentence-level approach

In [1]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.initializers import Constant
from gensim.models import KeyedVectors

RANDOM_STATE = 1212

cmu_data = pd.read_csv('cmu_data_final.csv')
imdb_data = pd.read_csv('imdb_data_final.csv')

# Dataset preparation

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize, remove stopwords, and lemmatize
    tokens = nltk.word_tokenize(text.lower())
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and (token.isalnum() or token == '.')]
    return ' '.join(filtered_tokens)

cmu_data['processed_plot'] = cmu_data['plot'].apply(preprocess_text)
imdb_data['processed_plot'] = imdb_data['plot'].apply(preprocess_text)

# Word2vec embeddings, trained on the CMU dataset
# word2vec_model = KeyedVectors.load('word2vec_model_from_cmu_utf8.bin')
# Google news word2vec embeddings
import gensim.downloader as api
word2vec_model = api.load("word2vec-google-news-300")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### CMU

In [3]:
# Tokenize and pad the text sequences
cmu_tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
cmu_tokenizer.fit_on_texts(cmu_data['processed_plot'])
cmu_sequences = cmu_tokenizer.texts_to_sequences(cmu_data['processed_plot'])
cmu_word_index = cmu_tokenizer.word_index
cmu_padded_sequences = pad_sequences(cmu_sequences, maxlen=300)

# Label encoding
cmu_data['genre'] = cmu_data['genre'].apply(lambda x: x.split('|'))
cmu_mlb = MultiLabelBinarizer()
cmu_genres_encoded = cmu_mlb.fit_transform(cmu_data['genre'])

# Train-test split
cmu_X_train, cmu_X_test, cmu_y_train, cmu_y_test = train_test_split(
    cmu_padded_sequences, cmu_genres_encoded, test_size=0.2, random_state=RANDOM_STATE)

### IMDb

In [4]:
# Tokenize and pad the text sequences
imdb_tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
imdb_tokenizer.fit_on_texts(imdb_data['processed_plot'])
imdb_sequences = imdb_tokenizer.texts_to_sequences(imdb_data['processed_plot'])
imdb_word_index = imdb_tokenizer.word_index
imdb_padded_sequences = pad_sequences(imdb_sequences, maxlen=300)

# Label encoding
imdb_data['genre'] = imdb_data['genre'].apply(lambda x: x.split('|'))
imdb_mlb = MultiLabelBinarizer()
imdb_genres_encoded = imdb_mlb.fit_transform(imdb_data['genre'])

# Train-test split
imdb_X_train, imdb_X_test, imdb_y_train, imdb_y_test = train_test_split(
    imdb_padded_sequences, imdb_genres_encoded, test_size=0.2, random_state=RANDOM_STATE)

# Model preparation

### CMU

In [5]:
# Create the word embedding matrix
""" cmu_embedding_dim = word2vec_model.vector_size
cmu_embedding_matrix = np.zeros((len(cmu_word_index) + 1, cmu_embedding_dim))
for word, i in cmu_word_index.items():
    if word in word2vec_model.wv:
        cmu_embedding_matrix[i] = word2vec_model.wv[word] """

# Create the word embedding matrix
cmu_embedding_dim = word2vec_model.vector_size
cmu_embedding_matrix = np.zeros((len(cmu_word_index) + 1, cmu_embedding_dim))
for word, i in cmu_word_index.items():
    if word in word2vec_model:
        cmu_embedding_matrix[i] = word2vec_model[word]

In [6]:
# Define and compile the LSTM model
model = Sequential()
model.add(Embedding(len(cmu_word_index) + 1, cmu_embedding_dim, embeddings_initializer=Constant(
    cmu_embedding_matrix), trainable=False))
model.add(LSTM(128, dropout=0.25, recurrent_dropout=0.25, return_sequences=True))
model.add(LSTM(64, dropout=0.25, recurrent_dropout=0.25))
model.add(Dense(len(cmu_mlb.classes_), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(cmu_X_train, cmu_y_train, batch_size=64, epochs=7, validation_data=(
    cmu_X_test, cmu_y_test))

Epoch 1/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 89s 750ms/step - accuracy: 0.3532 - loss: 1.3029 - val_accuracy: 0.5319 - val_loss: 1.0606
Epoch 2/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 52s 724ms/step - accuracy: 0.5292 - loss: 1.0951 - val_accuracy: 0.5860 - val_loss: 0.9742
Epoch 3/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 54s 751ms/step - accuracy: 0.5815 - loss: 1.0014 - val_accuracy: 0.6236 - val_loss: 0.9473
Epoch 4/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 53s 739ms/step - accuracy: 0.6017 - loss: 0.9677 - val_accuracy: 0.6384 - val_loss: 0.9216
Epoch 5/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 54s 757ms/step - accuracy: 0.6070 - loss: 0.9422 - val_accuracy: 0.6349 - val_loss: 0.9272
Epoch 6/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 55s 755ms/step - accuracy: 0.5958 - loss: 0.9637 - val_accuracy: 0.6332 - val_loss: 0.9102
Epoch 7/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 54s 746ms/step - accuracy: 0.6141 - loss: 0.9254 - val_accuracy: 0.6515 - val_loss: 0.8905


In [7]:
# Convert sequences back to text
cmu_X_train_reverted = cmu_tokenizer.sequences_to_texts(cmu_X_test)

sentencecount = 0
plotcount = 0
cmu_sentence_level_predictions = []

# Assuming X_train_reverted is a list of plot summaries
for plot_summary in cmu_X_train_reverted:
    # Split the plot summary into sentences based on "."
    sentences = [sentence.strip() for sentence in plot_summary.split(".") if sentence.strip()]
    num_sentences = len(sentences)
    # print("Number of sentences:", num_sentences)
    # print(plot_summary)
    # scores for each value in eg [0.24 0.65 0.22 0.12] (then devide it by number of sentences)
    score1 = 0
    score2 = 0
    score3 = 0
    score4 = 0
    
    for sentence in sentences:
        # print("Sentence loop beginning")
        # print(sentence)
        sequence = cmu_tokenizer.texts_to_sequences([sentence])
        padded_sequence = pad_sequences(sequence, maxlen=300)
        # print(padded_sequence[0])
        prediction = model.predict(padded_sequence)
        # print(prediction)
        #print(prediction[0][0])
        #print(prediction[0][1])
        #print(prediction[0][2])
        #print(prediction[0][3])
        score1 += prediction[0][0]
        score2 += prediction[0][1]
        score3 += prediction[0][2]
        score4 += prediction[0][3]
    plotcount += 1
    #print("Plots processed:", plotcount)
    processed_score1 = score1 / num_sentences
    processed_score2 = score2 / num_sentences
    processed_score3 = score3 / num_sentences
    processed_score4 = score4 / num_sentences
    processed_values = [processed_score1,processed_score2,processed_score3,processed_score4]
    #print('Processed values:')
    #print(processed_values)
    cmu_sentence_level_predictions.append(processed_values)
    print(plotcount)
#print("Total sentences processed:", sentencecount)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━

### IMDb

In [8]:
""" # Create the word embedding matrix
imdb_embedding_dim = word2vec_model.vector_size
imdb_embedding_matrix = np.zeros((len(imdb_word_index) + 1, imdb_embedding_dim))
for word, i in imdb_word_index.items():
    if word in word2vec_model.wv:
        imdb_embedding_matrix[i] = word2vec_model.wv[word] """

# Create the word embedding matrix
imdb_embedding_dim = word2vec_model.vector_size
imdb_embedding_matrix = np.zeros((len(imdb_word_index) + 1, imdb_embedding_dim))
for word, i in imdb_word_index.items():
    if word in word2vec_model:
        imdb_embedding_matrix[i] = word2vec_model[word]

In [9]:
# Define and compile the LSTM model
model = Sequential()
model.add(Embedding(len(imdb_word_index) + 1, imdb_embedding_dim, embeddings_initializer=Constant(
    imdb_embedding_matrix), trainable=False))
model.add(LSTM(128, dropout=0.25, recurrent_dropout=0.25, return_sequences=True))
model.add(LSTM(64, dropout=0.25, recurrent_dropout=0.25))
model.add(Dense(len(imdb_mlb.classes_), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(imdb_X_train, imdb_y_train, batch_size=64, epochs=7, validation_data=(
    imdb_X_test, imdb_y_test))

Epoch 1/7


72/72 ━━━━━━━━━━━━━━━━━━━━ 60s 713ms/step - accuracy: 0.3746 - loss: 1.2821 - val_accuracy: 0.6017 - val_loss: 0.9809
Epoch 2/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 59s 820ms/step - accuracy: 0.5900 - loss: 0.9969 - val_accuracy: 0.6393 - val_loss: 0.9329
Epoch 3/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 65s 901ms/step - accuracy: 0.6084 - loss: 0.9603 - val_accuracy: 0.6306 - val_loss: 0.9163
Epoch 4/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 61s 843ms/step - accuracy: 0.6284 - loss: 0.9166 - val_accuracy: 0.6341 - val_loss: 0.9073
Epoch 5/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 53s 734ms/step - accuracy: 0.6497 - loss: 0.8964 - val_accuracy: 0.6297 - val_loss: 0.9228
Epoch 6/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 54s 758ms/step - accuracy: 0.6488 - loss: 0.8883 - val_accuracy: 0.6349 - val_loss: 0.9142
Epoch 7/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 51s 703ms/step - accuracy: 0.6405 - loss: 0.8817 - val_accuracy: 0.6437 - val_loss: 0.8857


In [10]:
# Convert sequences back to text
imdb_X_train_reverted = imdb_tokenizer.sequences_to_texts(imdb_X_test)

sentencecount = 0
plotcount = 0
imdb_sentence_level_predictions = []

# Assuming X_train_reverted is a list of plot summaries
for plot_summary in imdb_X_train_reverted:
    # Split the plot summary into sentences based on "."
    sentences = [sentence.strip() for sentence in plot_summary.split(".") if sentence.strip()]
    num_sentences = len(sentences)
    
    # scores for each value in eg [0.24 0.65 0.22 0.12] (then devide it by number of sentences)
    score1 = 0
    score2 = 0
    score3 = 0
    score4 = 0
    
    for sentence in sentences:
        
        sequence = imdb_tokenizer.texts_to_sequences([sentence])
        padded_sequence = pad_sequences(sequence, maxlen=300)

        prediction = model.predict(padded_sequence)

        score1 += prediction[0][0]
        score2 += prediction[0][1]
        score3 += prediction[0][2]
        score4 += prediction[0][3]

    processed_score1 = score1 / num_sentences
    processed_score2 = score2 / num_sentences
    processed_score3 = score3 / num_sentences
    processed_score4 = score4 / num_sentences
    processed_values = [processed_score1,processed_score2,processed_score3,processed_score4]

    imdb_sentence_level_predictions.append(processed_values)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━

# Evaluation

## CMU

In [11]:
# Initialize an empty list to store the predicted classes
cmu_y_pred_classes_sentence = []

# Apply the threshold to each prediction
for prediction_values in cmu_sentence_level_predictions:
    # Convert the prediction values to a NumPy array
    prediction_values = np.array(prediction_values)
    
    # Apply the threshold and convert to int
    prediction_class = (prediction_values > 0.39).astype(int)
    
    # Append the prediction class to the list
    cmu_y_pred_classes_sentence.append(prediction_class)

# Convert the list of lists to a NumPy array
cmu_y_pred_classes_sentence = np.array(cmu_y_pred_classes_sentence)

In [12]:
# Calculate evaluation metrics
precision_micro = precision_score(cmu_y_test, cmu_y_pred_classes_sentence, average='micro')
recall_micro = recall_score(cmu_y_test, cmu_y_pred_classes_sentence, average='micro')
f1_micro = f1_score(cmu_y_test, cmu_y_pred_classes_sentence, average='micro')

precision_macro = precision_score(cmu_y_test, cmu_y_pred_classes_sentence, average='macro')
recall_macro = recall_score(cmu_y_test, cmu_y_pred_classes_sentence, average='macro')
f1_macro = f1_score(cmu_y_test, cmu_y_pred_classes_sentence, average='macro')

# Print the metrics
print(f'Micro Precision: {precision_micro}')
print(f'Micro Recall: {recall_micro}')
print(f'Micro F1-score: {f1_micro}')
print()
print(f'Macro Precision: {precision_macro}')
print(f'Macro Recall: {recall_macro}')
print(f'Macro F1-score: {f1_macro}')

Micro Precision: 0.5391812865497077
Micro Recall: 0.40262008733624455
Micro F1-score: 0.461

Macro Precision: 0.7559107764994603
Macro Recall: 0.4002116800606534
Macro F1-score: 0.41520839252760244


In [13]:
# Calculate evaluation metrics for each genre
cmu_genre_scores = {}
for i, genre in enumerate(cmu_mlb.classes_):
    genre_accuracy = accuracy_score(cmu_y_test[:, i], cmu_y_pred_classes_sentence[:, i])
    genre_precision = precision_score(cmu_y_test[:, i], cmu_y_pred_classes_sentence[:, i])
    genre_recall = recall_score(cmu_y_test[:, i], cmu_y_pred_classes_sentence[:, i])
    genre_f1 = f1_score(cmu_y_test[:, i], cmu_y_pred_classes_sentence[:, i])
    
    cmu_genre_scores[genre] = {'Accuracy': genre_accuracy,
                                'Precision': genre_precision,
                                'Recall': genre_recall,
                                'F1-score': genre_f1}

# Print scores for each genre
for genre, scores in cmu_genre_scores.items():
    print(f'Genre: {genre}')
    print(f'Accuracy: {scores["Accuracy"]}')
    print(f'Precision: {scores["Precision"]}')
    print(f'Recall: {scores["Recall"]}')
    print(f'F1-score: {scores["F1-score"]}')
    print()

Genre: Comedy
Accuracy: 0.7868995633187773
Precision: 0.9242424242424242
Recall: 0.20333333333333334
F1-score: 0.3333333333333333

Genre: Drama
Accuracy: 0.6602620087336245
Precision: 0.4240506329113924
Recall: 0.9146757679180887
F1-score: 0.5794594594594594

Genre: Horror
Accuracy: 0.8288209606986899
Precision: 0.891566265060241
Recall: 0.2835249042145594
F1-score: 0.43023255813953487

Genre: Thriller
Accuracy: 0.7825327510917031
Precision: 0.7837837837837838
Recall: 0.19931271477663232
F1-score: 0.3178082191780822



In [14]:
# count number of predictions of each genre
cmu_genre_counts = {}
cmu_classes_list = cmu_mlb.classes_.tolist()  # Convert numpy array to list
for genre in cmu_classes_list:
    genre_count = np.sum(cmu_y_pred_classes_sentence[:, cmu_classes_list.index(genre)])
    cmu_genre_counts[genre] = genre_count

# Print the genre counts
for genre, count in cmu_genre_counts.items():
    print(f'{genre}: {count}')

Comedy: 66
Drama: 632
Horror: 83
Thriller: 74


## IMDb

In [15]:
# Initialize an empty list to store the predicted classes
imdb_y_pred_classes_sentence = []

# Apply the threshold to each prediction
for prediction_values in imdb_sentence_level_predictions:
    # Convert the prediction values to a NumPy array
    prediction_values = np.array(prediction_values)
    
    # Apply the threshold and convert to int
    prediction_class = (prediction_values > 0.39).astype(int)
    
    # Append the prediction class to the list
    imdb_y_pred_classes_sentence.append(prediction_class)

# Convert the list of lists to a NumPy array
imdb_y_pred_classes_sentence = np.array(imdb_y_pred_classes_sentence)

In [16]:
# Calculate evaluation metrics
precision_micro = precision_score(imdb_y_test, imdb_y_pred_classes_sentence, average='micro')
recall_micro = recall_score(imdb_y_test, imdb_y_pred_classes_sentence, average='micro')
f1_micro = f1_score(imdb_y_test, imdb_y_pred_classes_sentence, average='micro')

precision_macro = precision_score(imdb_y_test, imdb_y_pred_classes_sentence, average='macro')
recall_macro = recall_score(imdb_y_test, imdb_y_pred_classes_sentence, average='macro')
f1_macro = f1_score(imdb_y_test, imdb_y_pred_classes_sentence, average='macro')

print(f'Precision (micro): {precision_micro}')
print(f'Recall (micro): {recall_micro}')
print(f'F1-score (micro): {f1_micro}')
print()
print(f'Precision (macro): {precision_macro}')
print(f'Recall (macro): {recall_macro}')
print(f'F1-score (macro): {f1_macro}')

Precision (micro): 0.7235984354628422
Recall (micro): 0.4847161572052402
F1-score (micro): 0.5805439330543933

Precision (macro): 0.7385562566037872
Recall (macro): 0.48461736545787126
F1-score (macro): 0.5714161551482573


In [17]:
# Calculate evaluation metrics for each genre
imdb_genre_scores = {}
for i, genre in enumerate(imdb_mlb.classes_):
    genre_accuracy = accuracy_score(imdb_y_test[:, i], imdb_y_pred_classes_sentence[:, i])
    genre_precision = precision_score(imdb_y_test[:, i], imdb_y_pred_classes_sentence[:, i])
    genre_recall = recall_score(imdb_y_test[:, i], imdb_y_pred_classes_sentence[:, i])
    genre_f1 = f1_score(imdb_y_test[:, i], imdb_y_pred_classes_sentence[:, i])
    
    imdb_genre_scores[genre] = {'Accuracy': genre_accuracy,
                                'Precision': genre_precision,
                                'Recall': genre_recall,
                                'F1-score': genre_f1}

# Print scores for each genre
for genre, scores in imdb_genre_scores.items():
    print(f'Genre: {genre}')
    print(f'Accuracy: {scores["Accuracy"]}')
    print(f'Precision: {scores["Precision"]}')
    print(f'Recall: {scores["Recall"]}')
    print(f'F1-score: {scores["F1-score"]}')
    print()

Genre: Comedy
Accuracy: 0.8375545851528384
Precision: 0.7375
Recall: 0.59
F1-score: 0.6555555555555556

Genre: Drama
Accuracy: 0.8043668122270743
Precision: 0.6342412451361867
Recall: 0.5563139931740614
F1-score: 0.5927272727272728

Genre: Horror
Accuracy: 0.862882096069869
Precision: 0.8132530120481928
Recall: 0.5172413793103449
F1-score: 0.6323185011709602

Genre: Thriller
Accuracy: 0.7947598253275109
Precision: 0.7692307692307693
Recall: 0.27491408934707906
F1-score: 0.4050632911392405



In [18]:
# count number of predictions of each genre
imdb_genre_counts = {}
classes_list = imdb_mlb.classes_.tolist()  # Convert numpy array to list
for genre in classes_list:
    genre_count = np.sum(imdb_y_pred_classes_sentence[:, classes_list.index(genre)])
    imdb_genre_counts[genre] = genre_count

# Print the genre counts
for genre, count in imdb_genre_counts.items():
    print(f'{genre}: {count}')

Comedy: 240
Drama: 257
Horror: 166
Thriller: 104
